In [1]:
import pandas as pd
from notebook_wrapper import data_selection

dfData = data_selection.getNotebookOutput()
dfData = dfData.loc[:, ~dfData.columns.str.contains("^Unnamed")]

with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(dfData.dtypes)

In [2]:
sum(dfData["akd"])/ len(dfData)

In [3]:
droppingColumns = ["subject_id", "hadm_id", "stay_id"]
categoryColumns = ["dka_type", "gender", "race", "liver_disease", "ckd_stage"]
labelColumn = "akd"
numericColumns = [
    col
    for col in dfData.columns
    if col not in droppingColumns + categoryColumns + [labelColumn]
    and dfData[col].dtype != "bool"
]

In [37]:
from matplotlib import pyplot as plt
import numpy as np
from sklearn.calibration import calibration_curve
from sklearn.metrics import auc, precision_score, roc_curve
from sklearn.model_selection import learning_curve


def plot_learning_curve(estimator, X, y):
    train_sizes, train_scores, test_scores = learning_curve( # type: ignore
        estimator, X, y, train_sizes=np.linspace(0.1, 1.0, 10), cv=5
    )

    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    plt.figure(figsize=(10, 6))
    plt.fill_between(
        train_sizes,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.1,
        color="r",
    )
    plt.fill_between(
        train_sizes,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.1,
        color="g",
    )
    plt.plot(train_sizes, train_scores_mean, "o-", color="r", label="Training score")
    plt.plot(
        train_sizes, test_scores_mean, "o-", color="g", label="Cross-validation score"
    )
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    plt.title("Learning Curve")
    plt.legend(loc="best")
    plt.grid()
    plt.show()


def plot_calibration_curve(estimator, X, y):
    prob_pos = estimator.predict_proba(X)[:, 1]
    fraction_of_positives, mean_predicted_value = calibration_curve(
        y, prob_pos, n_bins=10
    )

    plt.figure(figsize=(10, 6))
    plt.plot(mean_predicted_value, fraction_of_positives, "s-", label="Model")
    plt.plot([0, 1], [0, 1], "--", color="gray", label="Perfectly calibrated")
    plt.xlabel("Mean predicted probability")
    plt.ylabel("Fraction of positives")
    plt.title("Calibration Curve")
    plt.legend(loc="best")
    plt.grid()
    plt.show()


def classification_metric_evaluate(model, X, y, binary=True, Threshold=None):
    # Predict probabilities
    y_pred_proba = model.predict_proba(X)[:, 1]

    # For simplicity, let's assume the threshold is 0.5 if not provided
    if Threshold is None:
        Threshold = 0.5

    # Convert probabilities to binary predictions based on the threshold
    y_pred = (y_pred_proba >= Threshold).astype(int)

    # Calculate evaluation metrics
    fpr, tpr, _ = roc_curve(y, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    accuracy = accuracy_score(y, y_pred)
    precision = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)

    # Calculate confidence intervals for AUC
    from scipy import stats

    auc_values = np.array([auc for _, auc in sorted(zip(tpr, fpr))])  # Sort AUC values
    auc_l, auc_u = stats.t.interval(
        0.95, len(auc_values) - 1, loc=np.mean(auc_values), scale=stats.sem(auc_values)
    )

    # Construct dictionary to store evaluation metrics
    metric_dic = {
        "AUC": roc_auc,
        "AUC_L": auc_l,
        "AUC_U": auc_u,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "cutoff": Threshold,  # Include the threshold in the dictionary
    }

    # Return evaluation metrics and ROC curve points
    return fpr, tpr, metric_dic, y_pred

# Encode data

In [4]:
# Drop the first 3 columns (ids)
df = dfData.drop(columns=droppingColumns)

# Encode categorical variables
df = pd.get_dummies(df, columns=categoryColumns)

# Split train-test

In [6]:
from sklearn.model_selection import train_test_split


# Split data into features (X) and target variable (y)
X = df.drop(columns=["akd"])
y = df["akd"]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42, stratify=y
)

# Fill missing values

In [5]:
import pandas as pd
from sklearn.impute import KNNImputer


# fill missing value by k-nearest neighbors
imputer = KNNImputer(n_neighbors=5, weights="distance").fit(X_train)

X_train_filled = pd.DataFrame(imputer.transform(X_train), columns=(X_train.columns))
X_test_filled = pd.DataFrame(imputer.transform(X_test), columns=(X_test.columns))


# Plot curves 

In [27]:
import xgboost as xgb

clf = xgb.XGBClassifier()

plot_learning_curve(clf, X_train, y_train)

In [29]:
clf.fit(X_train, y_train)

In [30]:
plot_calibration_curve(clf, X_test, y_test)

In [38]:
from sklearn import clone
from sklearn.model_selection import StratifiedKFold

best_auc = 0.0
tprs_train, tprs_valid = [], []
fpr_train_alls, tpr_train_alls = [], []
mean_fpr = np.linspace(0, 1, 100)
KF = StratifiedKFold(n_splits=10, random_state=7, shuffle=True)
for i, (train_index, valid_index) in enumerate(KF.split(X_train, y_train)):
    X_train_kf, X_valid_kf = X_train.iloc[train_index], X_train.iloc[valid_index]
    Y_train_kf, Y_valid_kf = y_train.iloc[train_index], y_train.iloc[valid_index]

    model = clone(clf).fit(X_train_kf, Y_train_kf)

    fpr_train, tpr_train, metric_dic_train, _ = classification_metric_evaluate(
        model, X_train_kf, Y_train_kf,
    )
    fpr_valid, tpr_valid, metric_dic_valid, _ = classification_metric_evaluate(
        model, X_valid_kf, Y_valid_kf, Threshold=metric_dic_train["cutoff"]
    )
    metric_dic_valid.update({"cutoff": metric_dic_train["cutoff"]})

    if metric_dic_valid["AUC"] > best_auc:
        clf = model
        resThreshold = metric_dic_train["cutoff"]

    tprs_valid.append(np.interp(mean_fpr, fpr_valid, tpr_valid))
    tprs_valid[-1][0] = 0.0

    # To draw a picture, you only need plt.plot(fpr,tpr). The variable roc_auc just records the value of auc and calculates it through the auc() function.
    validationCurve =True
    if validationCurve:
        plt.plot(
            fpr_valid, tpr_valid,
            lw=1, alpha=0.4,
            label='ROC fold %4d (auc=%0.3f 95%%CI (%0.3f-%0.3f))' % (
            i + 1, metric_dic_valid['AUC'], metric_dic_valid['AUC_L'], metric_dic_valid['AUC_U']),
        )

    ##Training set ROC
    fpr_train_alls.append(fpr_train)
    tpr_train_alls.append(tpr_train)
    tprs_train.append(np.interp(mean_fpr, fpr_train, tpr_train))
    tprs_train[-1][0] = 0.0

# Cross-val with XGBoost

In [11]:
from sklearn.model_selection import cross_val_score
import xgboost as xgb


# Define parameters for XGBoost
params = {
    # "max_depth": 9,  # maximum depth of the tree
    # "colsample_bylevel": 1.0,
    # "colsample_bynode": 0.0,
    # "colsample_bytree": 0.23111437122462683,
    # "gamma": 0.0,
    # "learning_rate": 0.05,
    # "n_estimators": 1000,  # Set the number of boosting rounds
}

classifier = xgb.XGBClassifier(**params)

cv_scores = cross_val_score(classifier, X, y, cv=10, scoring="recall")


# Print the cross-validation scores
print("Cross-validation scores:", cv_scores)

# Calculate and print the mean accuracy
mean_accuracy = cv_scores.mean()
print("Mean accuracy:", mean_accuracy)

In [8]:
predictions = classifier.predict(X_test)
predictions

In [9]:
# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report


accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

print(classification_report(y_test, predictions))

In [17]:
import seaborn as sns
import sklearn.metrics as metrics
import matplotlib.pyplot as plt

cm = metrics.confusion_matrix(y_test, predictions)

# Plot confusion matrix
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=["Predicted Negative", "Predicted Positive"],
    yticklabels=["Actual Negative", "Actual Positive"],
)
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("Confusion Matrix")
plt.show()

# Hyperparams tune

In [18]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import make_scorer, f1_score, recall_score


# Define your XGBoost model
model = XGBClassifier()

# Define the parameter grid to search through
param_grid = {
    "max_depth": [8, 9, 10],  # Maximum tree depth
    "learning_rate": [0.1, 0.05, 0.01],  # Learning rate
    "n_estimators": [500, 800, 100],  # Number of trees
}

# Define a custom scoring function to optimize for minimizing false negatives
scorer = make_scorer(recall_score, pos_label=1)

# Perform Grid Search Cross Validation
grid_search = GridSearchCV(
    estimator=model, 
    param_grid=param_grid,
    # scoring=scorer,
    cv=5,
    verbose=1,
    n_jobs=-1
)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_params

In [14]:
# Train the model with the best parameters
best_model = XGBClassifier(**best_params)
best_model.fit(X_train, y_train)

# Make predictions on the test set
predictions = best_model.predict(X_test)

# Evaluate the model
f1 = f1_score(y_test, predictions)
print("F1 Score:", f1)

In [16]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

# LassoCV

In [16]:
from sklearn.linear_model import LassoCV
import pandas as pd

# Feature selection using LassoCV
lasso_cv = LassoCV(cv=5, random_state=7)
lasso_cv.fit(X, y)
selected_features = X.columns[lasso_cv.coef_ != 0]

with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(pd.DataFrame({"column_name": X.columns, "lasso_coef": lasso_cv.coef_}).T)

In [20]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB, ComplementNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier


# Splitting data into training and validation cohorts
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.15, random_state=42
)

# Define classifiers
classifiers = {
    "LogisticRegression": LogisticRegression(),
    "XGBClassifier": XGBClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
    "LGBMClassifier": LGBMClassifier(),
    "SVC": SVC(),
    "MLPClassifier": MLPClassifier(),
    "GaussianNB": GaussianNB(),
    "ComplementNB": ComplementNB(),
    "AdaBoostClassifier": AdaBoostClassifier(),
    "KNeighborsClassifier": KNeighborsClassifier(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "BaggingClassifier": BaggingClassifier(),
}

# Perform cross-validation and evaluate each model
results = {}
for clf_name, clf in classifiers.items():
    scores = cross_val_score(clf, X, y, cv=10)
    results[clf_name] = scores.mean()

# Print results
for clf_name, score in results.items():
    print(f"{clf_name}: {score}")

# Other

In [25]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import TargetEncoder
import xgboost as xgb

estimators = [
    ("encoder", TargetEncoder()),
    ("clf", xgb.XGBClassifier(random_state=8)),
]

pipe = Pipeline(steps=estimators, verbose=True)
pipe

In [29]:
from skopt import BayesSearchCV
from skopt.space import Real, Integer


searchSpace = {
    "clf__max_depth": Integer(5, 11),
    "clf__learning_rate": Real(0.001, 1.0, prior="log-uniform"),
    "clf__subsample": Real(0.5, 1.0),
    "clf__colsample_bytree": Real(0.0, 0.5),
    "clf__colsample_bylevel": Real(0.5, 1.0),
    "clf__colsample_bynode": Real(0.0, 0.5),
    "clf__reg_alpha": Real(0.0, 10.0),
    "clf__reg_lambda": Real(0.0, 10.0),
    "clf__gamma": Real(0.0, 10.0),
}

opt = BayesSearchCV(
    pipe, search_spaces=searchSpace, cv=3, n_iter=50, scoring="roc_auc", random_state=7
)

opt.fit(X, y)

In [32]:
opt.best_estimator_

In [30]:
opt.best_score_

In [31]:
opt.score(X_test, y_test)

In [ ]:
opt.best_estimator_.steps